In [21]:
import os
import numpy as np
import pandas as pd
import datetime
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [5]:
data_path = './data'

In [6]:
data = pd.read_csv(os.path.join(data_path,'events.csv'))
data.shape

(2756101, 5)

In [7]:
data.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


## Sessionization

In [10]:
def preprocess(data, logging=False):
    
    # convert unix time to pandas datetime
    data['date'] = pd.to_datetime(data['timestamp'], unit='ms', origin='unix')
    
    # convert event to categorical
    data['event_type'] = data['event'].astype('category')
    
    if logging:
        print('Range of transaction dates = ', data['date'].min(), 'to', data['date'].max())
        print('Number of unique visitor id = {:,}'.format(len(data['visitorid'].unique())))
        print('Number of unique item id = {:,}'.format(len(data['itemid'].unique())))
        print('Number of unique transaction id = {:,}'.format(len(data['transactionid'].unique())))
    
    return data

**attributes:** visitorid, startdate, enddate, viewitemid, addtocartitemid, transactionitemid

In [2]:
def sessionize(events_df: pd.DataFrame):

    session_duration = datetime.timedelta(minutes=30)
    gpby_visitorid = events_df.groupby('visitorid')

    session_list = []
    for a_visitorid in gpby_visitorid.groups:

        visitor_df = events_df.loc[gpby_visitorid.groups[a_visitorid], :].sort_values('date')
        if not visitor_df.empty:
            visitor_df.sort_values('date', inplace=True)

            # Initialise first session
            startdate = visitor_df.iloc[0, :]['date']
            visitorid = a_visitorid
            items_dict = dict([ (i, []) for i in events_df['event_type'].cat.categories ])
            for index, row in visitor_df.iterrows():

                # Check if current event date is within session duration
                if row['date'] - startdate <= session_duration:
                # Add itemid to the list according to event type (i.e. view, addtocart or transaction)
                    items_dict[row['event']].append(row['itemid'])
                    enddate = row['date']
                else:
                    # Complete current session
                    session_list.append([visitorid, startdate, enddate] + [ value for key, value in items_dict.items() ])
                    # Start a new session
                    startdate = row['date']
                    items_dict = dict([ (i, []) for i in events_df['event_type'].cat.categories ])
                    # Add current itemid
                    items_dict[row['event']].append(row['itemid'])

            # If dict if not empty, add item data as last session.
            incomplete_session = False
            for key, value in items_dict.items():
                if value:
                    incomplete_session = True
                    break
            if incomplete_session:
                session_list.append([visitorid, startdate, enddate] + [value for key, value in items_dict.items()])

    return session_list

In [14]:
out = preprocess(data[data.visitorid.isin([148130,1230504,339335])]).reset_index(drop=True)
display(out)

,timestamp,visitorid,event,itemid,transactionid,date,event_type
0,1433221204592,339335,view,82389,NaN,2015-06-02 05:00:04.592,view
1,1433220727879,339335,view,82389,NaN,2015-06-02 04:52:07.879,view
2,1434155689849,148130,view,434889,NaN,2015-06-13 00:34:49.849,view
3,1434396651617,148130,view,228638,NaN,2015-06-15 19:30:51.617,view
4,1434396644323,148130,view,228638,NaN,2015-06-15 19:30:44.323,view
5,1437524915470,1230504,view,339606,NaN,2015-07-22 00:28:35.470,view
6,1437593175700,1230504,view,339606,NaN,2015-07-22 19:26:15.700,view
7,1437592323927,1230504,view,339606,NaN,2015-07-22 19:12:03.927,view
8,1437592514657,1230504,addtocart,339606,NaN,2015-07-22 19:15:14.657,addtocart
9,1437593071424,1230504,transaction,339606,11255.0,2015-07-22 19:24:31.424,transaction


In [17]:
sessions_df = pd.DataFrame(sessionize(out), columns=['visitorid', 'startdate', 'enddate', 'addtocart', 'transaction', 'view'])
sessions_df

,visitorid,startdate,enddate,addtocart,transaction,view
0,148130,2015-06-13 00:34:49.849,2015-06-13 00:34:49.849,[],[],[434889]
1,148130,2015-06-15 19:30:44.323,2015-06-15 19:30:51.617,[],[],"[228638, 228638]"
2,148130,2015-07-26 21:06:47.416,2015-07-26 21:14:29.599,[],[],"[228638, 228638]"
3,339335,2015-06-02 04:52:07.879,2015-06-02 05:00:04.592,[],[],"[82389, 82389]"
4,1230504,2015-07-22 00:28:35.470,2015-07-22 00:28:35.470,[],[],[339606]
5,1230504,2015-07-22 19:12:03.927,2015-07-22 19:26:15.700,[339606],[339606],"[339606, 339606]"


## ItemID to CategoryID mapping

In [18]:
item_prop_1_df = pd.read_csv(os.path.join(data_path, 'item_properties_part1.csv'))
item_prop_2_df = pd.read_csv(os.path.join(data_path, 'item_properties_part2.csv'))

item_prop_df = pd.concat([item_prop_1_df, item_prop_2_df])
item_prop_df.reset_index(drop=True, inplace=True)

del item_prop_1_df
del item_prop_2_df

In [19]:
item_prop_df['date'] = pd.to_datetime(item_prop_df['timestamp'], unit='ms', origin='unix')
item_prop_df.loc[(item_prop_df['itemid'] == 25) & (item_prop_df['property'].str.contains('cat')), :].sort_values('timestamp')

,timestamp,itemid,property,value,date
7786530,1431226800000,25,categoryid,1509,2015-05-10 03:00:00
8382882,1431831600000,25,categoryid,1509,2015-05-17 03:00:00
8581666,1432436400000,25,categoryid,1509,2015-05-24 03:00:00
8979234,1433041200000,25,categoryid,1509,2015-05-31 03:00:00
7985314,1433646000000,25,categoryid,1509,2015-06-07 03:00:00
8184098,1434250800000,25,categoryid,1509,2015-06-14 03:00:00
8780450,1435460400000,25,categoryid,1509,2015-06-28 03:00:00
9575586,1436065200000,25,categoryid,1509,2015-07-05 03:00:00
9376802,1436670000000,25,categoryid,1509,2015-07-12 03:00:00
9178018,1437274800000,25,categoryid,1509,2015-07-19 03:00:00


In [20]:
item_prop_df.loc[(item_prop_df['property'].str.contains('cat') & (item_prop_df['value'] == '10')), :]

,timestamp,itemid,property,value,date
5195834,1435460400000,245380,categoryid,10,2015-06-28 03:00:00
6306471,1431831600000,449019,categoryid,10,2015-05-17 03:00:00
14930790,1431226800000,26377,categoryid,10,2015-05-10 03:00:00
16248942,1431226800000,96493,categoryid,10,2015-05-10 03:00:00


In [ ]:
# gby = item_prop_df.loc[item_prop_df['property'].str.contains('cat'), :].groupby('itemid')
# value_gby = item_prop_df.loc[item_prop_df['property'].str.contains('cat'), :].groupby('value')

Create a data frame of categoryid properties. This will be used to determine the categoryid of an itemid.

In [22]:
def itemtocat(item_prop_df):
    item_prop_df.sort_values(by='date', inplace=True)
    itemid_groups = item_prop_df.loc[item_prop_df['property'].str.contains('cat')].groupby('itemid')
    itemtocat_dict = {}

    for a_itemid, itemid_df in itemid_groups:
        cat_list = []
        # Get the categoryid for this itemid
        for v, grp_df in itemid_df.groupby([(itemid_df.value != itemid_df.value.shift()).cumsum()]):
            cat_list.append((grp_df.iloc[-1][4], grp_df.iloc[-1][3]))
        #Add itemid to dict,
        itemtocat_dict[a_itemid] = cat_list
    return itemtocat_dict

In [25]:
itemtocat_dict = itemtocat(item_prop_df[item_prop_df.itemid.isin(out.itemid.unique())])
itemtocat_dict[339606]

[(Timestamp('2015-05-10 03:00:00'), '279')]

In [26]:
def to_categoryid(itemid, enddate):
    catid = ''
    
    for time, cat in itemtocat_dict.get(itemid, []):
        if enddate <= time:
            catid = cat
            break
    if not catid:
        if itemtocat_dict.get(itemid, []):  # enddate did not match
            catid = itemtocat_dict[itemid][-1][1]  # set categiryid to last value
        else:
            catid = str(itemid) + '_no_cat_id'  # itemid not in dictionary
    return catid

In [27]:
{to_categoryid(i, sessions_df['enddate'][2]) for i in sessions_df['view'][2]}

{'819'}

## Feature engg.

In [49]:
def create_features(data):
    
    data.reset_index(drop=True, inplace=True)
    
    # pages
    data['pages'] = data['view'].apply(lambda x: len(x))
    
    # pagetime
    pages_more_than1 = data['pages'] > 1
    data.loc[pages_more_than1, 'pagetime'] = (data.loc[pages_more_than1, 'enddate'] - \
                                              data.loc[pages_more_than1, 'startdate']) / \
                                             (data.loc[pages_more_than1, 'pages'] - 1)
    
    pages_less_than1 = pages_more_than1.apply(lambda x: not x)
    data.loc[pages_less_than1, 'pagetime'] = pd.Timedelta(0)
    
    #convert page time to secs
    data.loc[:, 'pagetime'] = data.loc[:, 'pagetime'].map(pd.Timedelta.total_seconds)
    
    # change in category
    data['diffcat'] = data.loc[:, ['enddate', 'view']].apply(
    lambda x: len({ to_categoryid(i, x['enddate']) for i in x['view'] }) / len(x['view']) if x['view'] else 0, axis=1)
    #data['diffcat'] = data.loc[:, ['enddate', 'view']].apply(lambda x: (x.view), axis=1)
    #data.loc[[0, 1, 2], ['enddate', 'view']].apply(lambda x: print(x['view']), axis=1)
    
    # diffprod
    data['diffprod'] = data['view'].map(lambda x: len(set(x)) / len(x) if x else 0)
    
    # prodcat
    data['prodcat'] = data.loc[:, ['enddate', 'view']].apply(
    lambda x: len(set(x['view'])) / len({ to_categoryid(i, x['enddate']) for i in x['view'] }) if x['view'] else 0, axis=1)
    
    # maxrep
    data['maxrep'] = data['view'].map(lambda x: Counter(x).most_common(1)[0][1] if x else 0)
    
    return data

In [52]:
features_df = create_features(sessions_df)
features_df

,visitorid,startdate,enddate,addtocart,transaction,view,pages,diffcat,diffprod,prodcat,maxrep,pagetime
0,148130,2015-06-13 00:34:49.849,2015-06-13 00:34:49.849,[],[],[434889],1,1.0,1.0,1.0,1,0.000
1,148130,2015-06-15 19:30:44.323,2015-06-15 19:30:51.617,[],[],"[228638, 228638]",2,0.5,0.5,1.0,2,7.294
2,148130,2015-07-26 21:06:47.416,2015-07-26 21:14:29.599,[],[],"[228638, 228638]",2,0.5,0.5,1.0,2,462.183
3,339335,2015-06-02 04:52:07.879,2015-06-02 05:00:04.592,[],[],"[82389, 82389]",2,0.5,0.5,1.0,2,476.713
4,1230504,2015-07-22 00:28:35.470,2015-07-22 00:28:35.470,[],[],[339606],1,1.0,1.0,1.0,1,0.000
5,1230504,2015-07-22 19:12:03.927,2015-07-22 19:26:15.700,[339606],[339606],"[339606, 339606]",2,0.5,0.5,1.0,2,851.773


## Filtering

In [51]:
def filter_data(data):
    
    #remove sessions where no items were viewed, the visitor 
    # only performed addtocart or transact since these events
    # do not help to predict items added to cart
    data.drop(data.index[data['pages'] == 0], inplace=True)
    
    #remove the sessions where pagetime > 20 mins
    data.drop(data.index[data['pagetime'] > 1200], inplace=True)
    
    return data

In [54]:
filtered_df = filter_data(features_df)
filtered_df

,visitorid,startdate,enddate,addtocart,transaction,view,pages,diffcat,diffprod,prodcat,maxrep,pagetime
0,148130,2015-06-13 00:34:49.849,2015-06-13 00:34:49.849,[],[],[434889],1,1.0,1.0,1.0,1,0.000
1,148130,2015-06-15 19:30:44.323,2015-06-15 19:30:51.617,[],[],"[228638, 228638]",2,0.5,0.5,1.0,2,7.294
2,148130,2015-07-26 21:06:47.416,2015-07-26 21:14:29.599,[],[],"[228638, 228638]",2,0.5,0.5,1.0,2,462.183
3,339335,2015-06-02 04:52:07.879,2015-06-02 05:00:04.592,[],[],"[82389, 82389]",2,0.5,0.5,1.0,2,476.713
4,1230504,2015-07-22 00:28:35.470,2015-07-22 00:28:35.470,[],[],[339606],1,1.0,1.0,1.0,1,0.000
5,1230504,2015-07-22 19:12:03.927,2015-07-22 19:26:15.700,[339606],[339606],"[339606, 339606]",2,0.5,0.5,1.0,2,851.773


In [55]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [77]:
def prepare_data(data):
    
    # train-test split
    X_train, X_test = train_test_split(data.iloc[:, -6:].values.astype(float), test_size=.2, random_state=552)
    
    # scaling
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_std = scaler.transform(X_train)
    X_test_std = scaler.transform(X_test)
    
#     # k-means clustering
#     kmeans_list = [KMeans(n_clusters=i, max_iter=1000, random_state=435, n_jobs=-1) for i in range(2, 3)]
#     score = [kmeans_list[i].fit(X_train_std).score(X_train_std) for i in range(len(kmeans_list))]
    
#     cluster6_centers_df = pd.DataFrame.from_records(scaler.inverse_transform(kmeans_list[4].cluster_centers_), columns=sessions_df.columns[-6:])
#     cluster6_centers_df['size'] = np.bincount(kmeans_list[4].labels_)
    
#     cluster5_centers_df = pd.DataFrame.from_records(scaler.inverse_transform(kmeans_list[3].cluster_centers_), columns=sessions_df.columns[-6:])
#     cluster5_centers_df['size'] = np.bincount(kmeans_list[3].labels_)
    
#     cluster4_centers_df = pd.DataFrame.from_records(scaler.inverse_transform(kmeans_list[2].cluster_centers_), columns=sessions_df.columns[-6:])
#     cluster4_centers_df['size'] = np.bincount(kmeans_list[2].labels_)
    
#     model = kmeans_list[3]
#     data['cluster'] = model.predict(scaler.transform(data.iloc[:, -6:].values.astype('float')))
    
    return X_train_std, X_test_std, scaler

In [78]:
X_train_std, X_test_std, scaler = prepare_data(filtered_df)
display(X_train_std); display(X_test_std);

array([[ 0.57735027, -0.57735027, -0.57735027,  0.        ,  0.57735027,
        -0.9111247 ],
       [ 0.57735027, -0.57735027, -0.57735027,  0.        ,  0.57735027,
         1.4708617 ],
       [-1.73205081,  1.73205081,  1.73205081,  0.        , -1.73205081,
        -0.93169858],
       [ 0.57735027, -0.57735027, -0.57735027,  0.        ,  0.57735027,
         0.37196157]])

array([[ 0.57735027, -0.57735027, -0.57735027,  0.        ,  0.57735027,
         0.41294573],
       [-1.73205081,  1.73205081,  1.73205081,  0.        , -1.73205081,
        -0.93169858]])